In [2]:
import re
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
from pickle import dump, load
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# 文件路径列表
file_paths = [
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_1.jsonl',
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_2.jsonl',
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_3.jsonl',
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_4.jsonl',
    'D:\\学习\\Postgra\\Text Technologies for Data Science\\Project\\Query-Processing\\OneDrive_1_2024-2-16\\combined123\\combined_part_5.jsonl'
]

# 打开一个新文件用于写入所有标题
with open('all_titles.txt', 'w', encoding='utf-8') as out_file:
    # 遍历文件路径
    for file_path in file_paths:
        # 打开文件并逐行读取
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                # 将每行的内容从JSON字符串转换为字典
                article = json.loads(line)
                # 替换掉标题中的换行符'\n'，然后写入到输出文件中
                title = article['title'].replace('\n', ' ')
                out_file.write(title + '\n')


In [ ]:
def preprocess_text(text):
    """
    预处理文本：转换为小写并去除非字母数字字符
    """
    # 转换为小写
    text = text.lower()
    # 使用正则表达式去除非字母数字字符
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

# 输入文件路径
input_file_path = 'all_titles.txt'
# 输出文件路径
output_file_path = 'preprocessed_titles.txt'

# 读取输入文件，预处理每一行，然后写入到输出文件
with open(input_file_path, 'r', encoding='utf-8') as in_file, \
     open(output_file_path, 'w', encoding='utf-8') as out_file:
    for line in in_file:
        preprocessed_line = preprocess_text(line)
        out_file.write(preprocessed_line + '\n')


In [ ]:
file_path = 'preprocessed_titles.txt'

titles = []
with open(file_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        if i < 5000:  # 仅读取前5000行
            title = line.strip().lower()
            titles.append(title)
        else:
            break  # 当读取到5000行时，退出循环


tokenizer = Tokenizer()
tokenizer.fit_on_texts(titles)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in titles:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# 填充序列以保持统一长度
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# 分割为特征和标签
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = to_categorical(label, num_classes=total_words)

# 定义模型
def create_model(total_words, max_sequence_len):
    model = Sequential()
    model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))
    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
    return model

# 创建模型
model = create_model(total_words, max_sequence_len)
model.summary()

# 设置checkpoint
checkpoint_path = 'model_checkpoint2.h5'
checkpoint = ModelCheckpoint(checkpoint_path, monitor='loss', verbose=1, save_best_only=True, mode='min')

# 训练模型
model.fit(predictors, label, epochs=100, verbose=1, callbacks=[checkpoint],batch_size=64)

# 保存tokenizer
dump(tokenizer, open('tokenizer2.pkl', 'wb'))



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 45, 100)           900800    
                                                                 
 lstm (LSTM)                 (None, 45, 150)           150600    
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 9008)              909808    
                                                                 
Total params: 2,061,608
Trainable params: 2,061,608
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


D:\Toobox\Anaconda\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


393/393 [==============================] - ETA: 0s - loss: 9.3804 - accuracy: 0.0610
Epoch 1: loss improved from inf to 9.38040, saving model to model_checkpoint2.h5
393/393 [==============================] - 36s 82ms/step - loss: 9.3804 - accuracy: 0.0610
Epoch 2/100
393/393 [==============================] - ETA: 0s - loss: 8.8318 - accuracy: 0.0626
Epoch 2: loss improved from 9.38040 to 8.83185, saving model to model_checkpoint2.h5
393/393 [==============================] - 32s 81ms/step - loss: 8.8318 - accuracy: 0.0626
Epoch 3/100
 60/393 [===>..........................] - ETA: 26s - loss: 8.6700 - accuracy: 0.0729

In [ ]:
# 测试模型：
# 加载模型
model = load_model('model_checkpoint.h5')

# 加载tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))



In [ ]:
def generate_text(seed_text, next_words, model, tokenizer, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predictions = model.predict(token_list, verbose=0)
        predicted = np.argmax(predictions, axis=-1)[0]
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
seed_text = "Deep learning"
next_words = 10  # 生成标题的长度

generated_title = generate_text(seed_text, next_words, model, tokenizer, max_sequence_len)
print(generated_title)